# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below in order to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below in order to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        
 


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
    

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance with local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS music_event
                    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
                    """)

except Exception as e:
    print(e)
    

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music_event')

except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## QUESTION1
### Table sessions were created in order to query needs in question 1. The partition key which is session_id was used as a  partition key because it is filtering the data.

In [8]:

try:
    session.execute ("""CREATE TABLE IF NOT EXISTS sessions (session_id INT, item_in_session INT, artist TEXT, \
                        song_title TEXT, length FLOAT, PRIMARY KEY (session_id, item_in_session))""")

except Exception as e:
    print(e)              


In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO sessions (session_id, item_in_session, artist, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
       
        session.execute(query, (int(line[8]),int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query1 = "SELECT artist, song_title, length FROM sessions WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute (query1)

except Exception as e:
    print(e)

df = pd.DataFrame(rows._current_rows)
df


,artist,song_title,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## QUESTION2
### In table users, user_id, session_id were chosen as compound partition key because these are essential filtering limitations. Also, the item_in_session row has been chosen as a clustering key in order to sort the output in direction of the request.

In [11]:
try:
    session.execute ("""CREATE TABLE IF NOT EXISTS users (user_id INT, session_id INT, item_in_session INT,
                        song_title TEXT, \
                        artist TEXT, user_first_name TEXT, user_last_name TEXT, 
                        PRIMARY KEY ((user_id, session_id), item_in_session))""")

except Exception as e:
    print(e)                  
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (user_id, session_id, item_in_session, song_title, \
                 artist, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
       
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))     

query2 = "SELECT artist, item_in_session, song_title, user_first_name, user_last_name FROM users \
          WHERE user_id = 10 AND session_id = 182"
try:
    result = session.execute (query2)

except Exception as e:
    print(e)
df = pd.DataFrame(result._current_rows)
df


,artist,item_in_session,song_title,user_first_name,user_last_name
0,Down To The Bone,0,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,1,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,2,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,3,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## QUESTION3 
### specific_song collection has been edited to extract data as requested in question 3. the song line was not enough to filter the data. Thus, user_id was also used as a partition key.

In [12]:
try:
    session.execute ("""CREATE TABLE IF NOT EXISTS specific_song (song TEXT, user_id INT, user_first_name TEXT, 
                     user_last_name TEXT, PRIMARY KEY (song, user_id))""")

except Exception as e:
    print(e)              

    file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO specific_song (song, user_id, user_first_name, user_last_name)"
        query = query + " VALUES ( %s, %s, %s, %s)"
        
        session.execute (query, (line[9],int(line[10]), line[1], line[4]))  

query3 = "SELECT user_first_name, user_last_name FROM specific_song WHERE song = 'All Hands Against His Own'"
                         
try:
    rows = session.execute (query3)

except Exception as e:
    print(e)


df = pd.DataFrame(rows._current_rows)
df         


,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Dropping the tables before closing out the sessions

In [13]:
try:
    session.execute ("""DROP TABLE IF EXISTS sessions""")
    session.execute ("""DROP TABLE IF EXISTS specific_song""")
    session.execute ("""DROP TABLE IF EXISTS users""")
except Exception as e:
    print(e)
    
   

### Closing the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()